# WEB SCRAPING 

En este notebook se encuentra el código usado para realizar el escaneo y extracción de información de distintas páginas web para conformar el dataset de precios de vivienda en el Valle de San Nicolas, finalmente se decide usar unicamente los datos obtenidos del sitio Finca Raíz.

www.fincaraiz.com

In [ ]:
!pip install selenium
!pip install beautifulsoup4
!pip install lxml

In [ ]:
#!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (91.0.4472.101-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup, Comment
from selenium.webdriver import ActionChains
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
from progressbar import progressbar as pbar

In [ ]:
# Generar instancia de selenium para realizar el scraping
driver = webdriver.Chrome(chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


## **Información Página Finca Raiz**

***Capturar URLS de finca raiz***

In [ ]:
# Capturar datos de finca raiz
#driver = webdriver.Chrome('D:/chromedriver',chrome_options=chrome_options)
driver = webdriver.Chrome(chrome_options=chrome_options)


URL="https://google.com.co"
driver.get(URL)
driver.maximize_window()
time.sleep(1)

sopas=[]

#Hay 4057 resultados lo que da aprox 91 paginas
#Inicialmente se navega en la URL de búsqueda de finca raíz para capturar todas las URLs de los resultados deseados, posterior a esto,
#se hace scraping sobre cada una de las paaginas para extraer los datos de interés

for i in range(91):
             
    URL2=f"https://www.fincaraiz.com.co/apartamento-casa-finca-casa-campestre-apartaestudio-casa-lote/venta/rionegro-el-retiro-la-ceja-guarne-el-carmen-de-viboral-marinilla-llano-grande-san-antonio-de-pereira-santuario/?ad=30|{i}||||1||8,9,7,21,22,23|||55|5500004,5500013,5500012,5500018,5500026,5500024,5500020,5500022,5500029,5500028||||||||||||||||1|||1||griddate%20desc||||-1||||"   

    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL+"t");
    driver.get(URL2);
    #time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sopas.append(soup)
    
    #Cerrar la pestaña actual
    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL + "w");
    
    print(f"Páginas escaneadas:{i+1}")

driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


Páginas escaneadas:1
Páginas escaneadas:2
Páginas escaneadas:3
Páginas escaneadas:4
Páginas escaneadas:5
Páginas escaneadas:6
Páginas escaneadas:7
Páginas escaneadas:8
Páginas escaneadas:9
Páginas escaneadas:10
Páginas escaneadas:11
Páginas escaneadas:12
Páginas escaneadas:13
Páginas escaneadas:14
Páginas escaneadas:15
Páginas escaneadas:16
Páginas escaneadas:17
Páginas escaneadas:18
Páginas escaneadas:19
Páginas escaneadas:20
Páginas escaneadas:21
Páginas escaneadas:22
Páginas escaneadas:23
Páginas escaneadas:24
Páginas escaneadas:25
Páginas escaneadas:26
Páginas escaneadas:27
Páginas escaneadas:28
Páginas escaneadas:29
Páginas escaneadas:30
Páginas escaneadas:31
Páginas escaneadas:32
Páginas escaneadas:33
Páginas escaneadas:34
Páginas escaneadas:35
Páginas escaneadas:36
Páginas escaneadas:37
Páginas escaneadas:38
Páginas escaneadas:39
Páginas escaneadas:40
Páginas escaneadas:41
Páginas escaneadas:42
Páginas escaneadas:43
Páginas escaneadas:44
Páginas escaneadas:45
Páginas escaneadas:

In [ ]:
# los proyectos nuevos tienen una estructura diferente que los usados en la página de detalle 
# A través de Beatiful soup se realiza una transformación del código html de los sitios para extraer los datos a 
# partir de etiquetas, clases y ids de los elementos de la página web

paginas_prop=[]

for soup in sopas:

    propiedades = soup.find_all('ul', class_='advert')

    for data in propiedades:
        url=data.a.get("href")

        if url.find("https://") == -1:
            url="https://www.fincaraiz.com.co"+url

        paginas_prop.append(url)

In [ ]:

df_paginas_prop = pd.DataFrame (paginas_prop,columns=['URLS'])

# Guardar lista de páginas 
df_paginas_prop.to_csv('PAGINAS_act_08.08.2021.csv', header=True, index=False, sep=';', encoding='latin1')

In [ ]:
# Función para quitar comentarios de un documento html, ya que esto genera errores al momento de extraer datos
def quitar_comentarios(datos):
  for element in datos(text=lambda text: isinstance(text, Comment)):
      element.extract()

In [ ]:
#paginas_prop=['https://www.fincaraiz.com.co/avanti/el-retiro/proyecto-nuevo-det-4283005.aspx?itemid=4283008','https://www.fincaraiz.com.co/altos-de-santana/rionegro/proyecto-nuevo-det-6351568.aspx?itemid=6357639']

In [ ]:
#Limpiar urls duplicadas

import urllib.parse as up
from urllib.parse import parse_qs

clean_url=[]

for url in paginas_prop:
    
    if url.find("https://logs1237.xiti.com/") != -1:
        parsed =  up.urlparse(url)
        parsed= parse_qs(parsed.query)['url'][0]
        clean=parsed.replace('?itemid=','').replace(parse_qs(up.urlparse(parsed).query)['itemid'][0],'')
        clean_url.append(clean)
    else:
        clean_url.append(url)
        
paginas_prop=list(dict.fromkeys(clean_url))

In [ ]:
#Leer una a una la página de las propiedades y extraer información en un array de diccionarios

data_detalle=[]

def run_session(url):
    
    driver = webdriver.Chrome(chrome_options=chrome_options)

    URL="https://google.com.co"
    driver.get(URL)
    driver.maximize_window()

    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL+"t");
    driver.get(url);

    soup = BeautifulSoup(driver.page_source, 'lxml')

    datos_apartamentos = soup.find_all("div", {"id":"typology"})

    script_data = driver.execute_script('return sfAdvert')

    info={}
    
    #Es propiedad usada
    if not datos_apartamentos:

        info["tipo"]='Usada'
        info["url"]=url
        info["titulo"]=soup.find(class_='title').find(class_='box').h1.text.replace('\n','').strip()
        info["precio"]=soup.find(class_='price').h2.text.replace('\n','').strip()
        info["area"]=soup.find(class_='features').find(class_='advertSurface').text.replace('\n','').strip()
        info["habitaciones"]=soup.find(class_='features').find(class_='advertRooms').text.replace('\n','').replace('Habitaciones:','').strip()
        info["banos"]=soup.find(class_='features').find(class_='advertBaths').text.replace('\n','').replace('Baños:','').strip()
        info["garajes"]=soup.find(class_='features').find(class_='advertGarages').text.replace('\n','').replace('Parqueaderos:','').strip()
        info["descripcion"] = soup.find(class_="description").p.text

        try:
            caractint = soup.find("ul", {"id":"tblInitialInteriores"})
            caractint_clean=[caract.text for caract in caractint]
            for carac in caractint_clean:
                info[carac]=1
        except:
            caractint_clean=[]

        try:
            caractext = soup.find("ul", {"id":"tblInitialExteriores"})
            caractext_clean=[caract.text for caract in caractext]
            for carac in caractext_clean:
                info[carac]=1
        except:        
            caractext_clean=[]

        try:
            caractsec = soup.find("ul", {"id":"tblInitialdelSector"})
            caractsec_clean=[caract.text for caract in caractsec]
            for carac in caractsec_clean:
                info[carac]=1

        except:
            caractsec_clean=[]

        info["caractint"] = caractint_clean
        info["caractext"] = caractext_clean
        info["caractsec"] = caractsec_clean

        otros_datos = soup.find(class_='features_2').ul
        quitar_comentarios(otros_datos)

        otros_datos = [dato.text for dato in otros_datos if dato != '\n']

        datos_estrato = [dato.replace('\n','').replace('Estrato:','').strip() for dato in otros_datos if dato.find("Estrato") != -1 ]
        info["estrato"] = datos_estrato[0] if len(datos_estrato)>0  else ''

        datos_antigu = [dato.replace('\n','').replace('Antigüedad:','').strip() for dato in otros_datos if dato.find("Antigüedad") != -1 ]
        info["antiguedad"] =  datos_antigu[0] if len(datos_antigu)>0  else ''

        try:
          info["latitud"] = script_data["latitude"]
          info["longitud"] = script_data["longitude"]
        except:
          info["latitud"] = 0
          info["longitud"] = 0

        data_detalle.append(info)

    #Es propiedad nueva
    else:

        datos_apartamentos = soup.find("div", {"id":"typology"}).table.tbody

        array_apa=[]

        for apt in datos_apartamentos:
            info_apartamento=[caract.text.replace('\n','').strip() for caract in apt if caract != '\n']

        if len(info_apartamento)>0:
            array_apa.append(info_apartamento)

        for apartamento in array_apa:

            
            info["tipo"]='Nueva'
            info["url"]=url
            info["titulo"]= soup.find(class_='title').h1.text.replace('\n','') +' ' + soup.find(class_='title').span.text.replace('\n','').strip() 
            info["precio"]= apartamento[6] if len(apartamento[6])>0  else  soup.find(class_='price').h2.text.replace('\n','').strip()
            info["area"]= apartamento[2] if len(apartamento[2])>0  else soup.find(class_='features').find(class_='advertSurface').text.replace('\n','').strip()
            info["habitaciones"]=apartamento[4] if len(apartamento[4])>0  else soup.find(class_='features').find(class_='advertRooms').text.replace('\n','').replace('Habitaciones:','').strip()
            info["banos"]= apartamento[5] if len(apartamento[5])>0  else soup.find(class_='features').find(class_='advertBaths').text.replace('Baños:','').strip()
            info["garajes"]=soup.find(class_='features').find(class_='advertGarages').text.replace('\n','').replace('Parqueaderos:','').strip()
            info["descripcion"] = soup.find(class_="description").b.text + ' ' + soup.find(class_="description").p.text
            info["antiguedad"] = 0


        try:
            caractint = soup.find("ul", {"id":"tblInitialInteriores"})
            caractint_clean=[caract.text for caract in caractint]
            for carac in caractint_clean:
                info[carac]=1
        except:
            caractint_clean=[]

        try:
            caractext = soup.find("ul", {"id":"tblInitialExteriores"})
            caractext_clean=[caract.text for caract in caractext]
            for carac in caractext_clean:
                info[carac]=1

        except:        
            caractext_clean=[]

        try:
            caractsec = soup.find("ul", {"id":"tblInitialdelSector"})
            caractsec_clean=[caract.text for caract in caractsec]
            for carac in caractsec_clean:
                info[carac]=1

        except:
            caractsec_clean=[]

        info["caractint"] = caractint_clean
        info["caractext"] = caractext_clean
        info["caractsec"] = caractsec_clean
        info["otros_datos"] = script_data

        try:
          info["latitud"] = script_data["latitude"]
          info["longitud"] = script_data["longitude"]
        except:
          info["latitud"] = 0
          info["longitud"] = 0

        data_detalle.append(info)
        
        return info

    #Cerrar la pestaña actual
    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL + "w");

    print(f"Página escaneada")

    driver.close() 



In [ ]:
from joblib import Parallel, delayed

from tqdm import tqdm

data_result = Parallel(n_jobs=-1)(delayed(run_session)(url) for url in tqdm(paginas_prop))

In [ ]:
run_session(paginas_prop[1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


Página escaneada


In [ ]:
data_detalle

[{'Acceso Pavimentado': 1,
  'Asador': 1,
  'Cancha de Baloncesto': 1,
  'Cancha de Futbol': 1,
  'Cerca de Zona Urbana': 1,
  'Chimenea': 1,
  'Cocina Equipada': 1,
  'Cocina Integral': 1,
  'Kiosko': 1,
  'Pesebrera': 1,
  'Sauna / Turco / Jacuzzi': 1,
  'Servicios Públicos': 1,
  'Trans. Público cercano': 1,
  'Ventilación Natural': 1,
  'Zona de BBQ': 1,
  'antiguedad': '1 a 8 años',
  'area': '600,00 m²',
  'banos': '6',
  'caractext': ['Acceso Pavimentado',
   'Árboles frutales',
   'Asador',
   'Cancha de Baloncesto',
   'Cancha de Futbol',
   'Kiosko',
   'Pesebrera'],
  'caractint': ['Chimenea',
   'Cocina Equipada',
   'Cocina Integral',
   'Sauna / Turco / Jacuzzi',
   'Servicios Públicos',
   'Ventilación Natural',
   'Zona de BBQ'],
  'caractsec': ['Cerca de Zona Urbana', 'Trans. Público cercano'],
  'descripcion': 'ESPECTACULAR CASA EN LLANOGRANDE EN UN SECTOR MUY TRANQUILO Y CERCANO A TODO,  CUENTA CON HERMOSOS ACABADOS , TOPOGRAFÍA PLANA APROVECHABLE Y EXCELENTES VÍAS D

In [ ]:
import pandas as pd
df_propiedades = pd.DataFrame(data_detalle)


In [ ]:
#df_propiedades=[]

In [ ]:
print(f'Número de registros: {df_propiedades.shape[0]}')

print(f'Número de Columnas: {df_propiedades.shape[1]}')

print('Columnas:')

#list(df_propiedades.columns)

Número de registros: 0
Número de Columnas: 0
Columnas:


In [ ]:
#Guardar datos en rchivo csv para uso posterior
df_propiedades.to_csv('consolidado_propiedades_FR_29052021.csv', header=True, index=False, sep=';', encoding='latin1')